In [1]:
import requests
import json
import random
import time
import pickle


model = "dRIT"

template = {
  "Answer": "",
  "Score Lexical Semantics": "",
  "Score Predicate-Argument Structure": "",
  "Score Logic": "",
  "Score Knowledge & Common Sense": ""
}


# TRAIN de MultiNLI

In [3]:
import pandas as pd

In [4]:
df =pd.read_pickle("predictions/RL_MultiNLI_DEVM.pickle")
df

,relation,no_matcheadas,contradiction,jaccard,simBoW,sumas,mutinf_t,max_info_t,entail,list_comp,list_incomp,Jaro-Winkler_rit,negT,negH,overlap_ent,clases
0,2,1.000000,0.0,0.000000,0.333333,0.163929,4.064680,0.446530,0.750000,2,0,0.000000,0,0,0.000000,neutral
1,2,0.666667,0.0,0.333333,0.402015,0.068591,4.161646,0.597851,0.750000,3,0,0.424242,0,1,0.333333,contradiction
2,2,0.600000,0.0,0.400000,0.226455,0.031096,4.187550,0.608355,0.714286,2,0,0.338384,1,0,0.400000,entailment
3,2,0.666667,0.0,0.000000,0.192450,0.037353,4.240373,0.567515,0.444444,1,0,0.515873,0,0,0.333333,contradiction
4,-1,0.000000,1.0,1.000000,0.258199,0.097872,4.245573,0.729875,0.500000,0,0,0.511111,1,0,0.000000,contradiction
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9892,2,0.428571,0.0,0.500000,0.408248,0.034530,4.332535,0.634045,0.625000,1,0,0.354377,0,0,0.571429,neutral
9893,1,0.000000,0.0,0.750000,0.717137,0.135855,4.512145,0.847485,0.833333,0,0,0.819048,0,0,1.000000,contradiction
9894,2,0.400000,0.0,0.500000,0.357771,0.123006,4.346858,0.805978,1.000000,1,0,0.585965,0,0,0.600000,entailment
9895,1,0.000000,0.0,0.500000,0.552052,0.115314,4.741702,1.000000,1.000000,0,0,0.740741,0,0,1.000000,entailment


In [9]:
original =pd.read_csv("../OPENAI/data/MultiNLI/DEVM/DEV_1.csv")
original

,sentence1,sentence2,gold_label
0,The new rights are nice enough,Everyone really likes the newest benefits,neutral
1,This site includes a list of all award winners...,The Government Executive articles housed on th...,contradiction
2,uh i don't know i i have mixed emotions about ...,"I like him for the most part, but would still ...",entailment
3,yeah i i think my favorite restaurant is alway...,My favorite restaurants are always at least a ...,contradiction
4,i don't know um do you do a lot of camping,I know exactly.,contradiction
...,...,...,...
9892,"Since 1998, LSC has initiated and overseen sig...",LSC has been focusing on improving it's state ...,neutral
9893,Eighty percent of pagers in the United States ...,Pagers in the United States were unaffected by...,contradiction
9894,"Finally, the FDA will conduct workshops, issue...",The FDA is set to conduct workshops.,entailment
9895,Cirque du Soleil's The latest from the acclaim...,Cirque du Soleil is an international troupe.,entailment


In [10]:
df["sentence_A"]=original["sentence1"]
df["sentence_B"]=original["sentence2"]

In [11]:
entailment=df[df["clases"]=="entailment"]
neutral=df[df["clases"]=="neutral"]
contradiction=df[df["clases"]=="contradiction"]
contradiction

,relation,no_matcheadas,contradiction,jaccard,simBoW,sumas,mutinf_t,max_info_t,entail,list_comp,list_incomp,Jaro-Winkler_rit,negT,negH,overlap_ent,clases,sentence_A,sentence_B
1,2,0.666667,0.0,0.333333,0.402015,0.068591,4.161646,0.597851,0.750000,3,0,0.424242,0,1,0.333333,contradiction,This site includes a list of all award winners...,The Government Executive articles housed on th...
3,2,0.666667,0.0,0.000000,0.192450,0.037353,4.240373,0.567515,0.444444,1,0,0.515873,0,0,0.333333,contradiction,yeah i i think my favorite restaurant is alway...,My favorite restaurants are always at least a ...
4,-1,0.000000,1.0,1.000000,0.258199,0.097872,4.245573,0.729875,0.500000,0,0,0.511111,1,0,0.000000,contradiction,i don't know um do you do a lot of camping,I know exactly.
5,2,0.500000,0.0,0.250000,0.267261,0.034874,4.306696,0.701959,0.600000,1,1,0.388889,0,0,0.500000,contradiction,well that would be a help i wish they would do...,We have plenty of space in the landfill.
6,2,0.333333,0.0,0.666667,0.577350,0.317886,4.547776,0.851813,0.750000,0,0,0.527778,0,1,0.666667,contradiction,yeah i know and i did that all through college...,I did that all through college but it never wo...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9881,2,0.666667,0.0,0.500000,0.408248,0.145363,4.318213,0.712210,0.800000,1,0,0.611111,0,0,0.333333,contradiction,other side that's a good idea,Doing the other side is a terrible idea.
9883,1,0.000000,0.0,0.000000,0.436436,0.093955,4.205127,0.598945,0.333333,0,1,0.492063,0,0,1.000000,contradiction,The information provided in this guide is curr...,The guide is outdated.
9891,2,0.750000,0.0,0.000000,0.316228,0.034208,4.205771,0.528781,0.833333,3,0,0.508333,1,0,0.250000,contradiction,"Second, reducing the rate of HIV transmission ...",No one cares about HIV or any other social pro...
9893,1,0.000000,0.0,0.750000,0.717137,0.135855,4.512145,0.847485,0.833333,0,0,0.819048,0,0,1.000000,contradiction,Eighty percent of pagers in the United States ...,Pagers in the United States were unaffected by...


In [12]:
df_e=entailment.sample(n=200)
df_n=neutral.sample(n=200)
df_c=contradiction.sample(n=200)
df_t=pd.concat([df_e,df_n,df_c])
df_t

,relation,no_matcheadas,contradiction,jaccard,simBoW,sumas,mutinf_t,max_info_t,entail,list_comp,list_incomp,Jaro-Winkler_rit,negT,negH,overlap_ent,clases,sentence_A,sentence_B
8091,2,0.333333,0.000000,0.333333,0.485071,0.163474,4.527890,0.850460,1.000000,1,0,0.650000,1,1,0.666667,entailment,"""So your girl writes that your little farewel...",Your farewell activity didn't go well.
5503,2,0.200000,0.000000,1.000000,0.615457,0.149428,4.527443,0.879779,1.000000,1,0,0.590909,0,0,0.800000,entailment,"Under Deng Xiaoping, Beijing actively sought t...",Beijing sought to create a good relationship.
1282,2,0.250000,0.000000,0.500000,0.535303,0.038836,4.411816,0.731585,0.888889,2,0,0.497076,0,0,0.750000,entailment,GAO also issued over 160 reports detailing spe...,Improving government readiness was aided by th...
7237,1,0.000000,0.000000,1.000000,1.000000,0.340767,4.654967,1.000000,1.000000,0,0,1.000000,0,0,1.000000,entailment,The chain wielder smiled at her.,The chain wielder was smiling.
1540,2,0.600000,0.000000,0.500000,0.353553,0.075418,4.190316,0.550815,0.800000,2,0,0.563889,0,0,0.400000,entailment,The celebrity-obsessed magazine surpasses itse...,The magazine publishes a lot of stories on cel...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6875,2,1.000000,0.000000,0.000000,0.198030,0.011191,3.987838,0.441345,0.600000,2,0,0.408497,0,1,0.000000,contradiction,There are also a couple of small aircraft lyin...,Abandoned aircraft have never been found in th...
4443,2,1.000000,0.000000,0.000000,0.182574,0.090118,3.795031,0.510137,1.000000,1,0,0.000000,0,0,0.000000,contradiction,"""The summons was only for Dave Hanson,"" Ser P...",All of them had been invited.
4662,-1,0.333333,0.333333,0.333333,0.306186,0.147137,4.739438,1.000000,1.000000,0,0,0.000000,0,0,0.333333,contradiction,The analysis also addresses the various altern...,There are no standards for design.
1436,1,0.000000,0.000000,0.666667,0.730297,0.047815,4.365051,0.698024,0.600000,0,0,0.822222,0,0,1.000000,contradiction,Scotland became little more than an English co...,Scotland was far greater than an English county.


In [13]:
lista_respuestasOllama=[]

In [14]:
df.columns

Index(['relation', 'no_matcheadas', 'contradiction', 'jaccard', 'simBoW',
       'sumas', 'mutinf_t', 'max_info_t', 'entail', 'list_comp', 'list_incomp',
       'Jaro-Winkler_rit', 'negT', 'negH', 'overlap_ent', 'clases',
       'sentence_A', 'sentence_B'],
      dtype='object')

In [15]:
faltantes=[]

In [16]:
df_t=df_t.reset_index(drop=True)
df_t

,relation,no_matcheadas,contradiction,jaccard,simBoW,sumas,mutinf_t,max_info_t,entail,list_comp,list_incomp,Jaro-Winkler_rit,negT,negH,overlap_ent,clases,sentence_A,sentence_B
0,2,0.333333,0.000000,0.333333,0.485071,0.163474,4.527890,0.850460,1.000000,1,0,0.650000,1,1,0.666667,entailment,"""So your girl writes that your little farewel...",Your farewell activity didn't go well.
1,2,0.200000,0.000000,1.000000,0.615457,0.149428,4.527443,0.879779,1.000000,1,0,0.590909,0,0,0.800000,entailment,"Under Deng Xiaoping, Beijing actively sought t...",Beijing sought to create a good relationship.
2,2,0.250000,0.000000,0.500000,0.535303,0.038836,4.411816,0.731585,0.888889,2,0,0.497076,0,0,0.750000,entailment,GAO also issued over 160 reports detailing spe...,Improving government readiness was aided by th...
3,1,0.000000,0.000000,1.000000,1.000000,0.340767,4.654967,1.000000,1.000000,0,0,1.000000,0,0,1.000000,entailment,The chain wielder smiled at her.,The chain wielder was smiling.
4,2,0.600000,0.000000,0.500000,0.353553,0.075418,4.190316,0.550815,0.800000,2,0,0.563889,0,0,0.400000,entailment,The celebrity-obsessed magazine surpasses itse...,The magazine publishes a lot of stories on cel...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,2,1.000000,0.000000,0.000000,0.198030,0.011191,3.987838,0.441345,0.600000,2,0,0.408497,0,1,0.000000,contradiction,There are also a couple of small aircraft lyin...,Abandoned aircraft have never been found in th...
596,2,1.000000,0.000000,0.000000,0.182574,0.090118,3.795031,0.510137,1.000000,1,0,0.000000,0,0,0.000000,contradiction,"""The summons was only for Dave Hanson,"" Ser P...",All of them had been invited.
597,-1,0.333333,0.333333,0.333333,0.306186,0.147137,4.739438,1.000000,1.000000,0,0,0.000000,0,0,0.333333,contradiction,The analysis also addresses the various altern...,There are no standards for design.
598,1,0.000000,0.000000,0.666667,0.730297,0.047815,4.365051,0.698024,0.600000,0,0,0.822222,0,0,1.000000,contradiction,Scotland became little more than an English co...,Scotland was far greater than an English county.


In [17]:
df_t.to_pickle("predictions/RL_MultiNLI_DEVM_600.pickle")

# RIT base

In [18]:
for i in range(1):
    for index,strings in df_t[i*600:(i+1)*600].iterrows():
        prompt = ''''Analyze the following sentences: 
        Text: '''+strings["sentence_A"]+'''
        Hypothesis: '''+ strings["sentence_B"]+'''
        only responds using the template. '''
        #print(prompt)
        data = {
            "prompt": prompt,
            "model": model,
            "format": "json",
            "stream": False,
            #"options": {"temperature": 0.2},
        }
        try:
            response = requests.post("http://localhost:11434/api/generate", json=data, stream=False,timeout=15)        
            json_data = json.loads(response.text)
            lista_respuestasOllama.append(json.dumps(json.loads(json_data["response"]), indent=2))
            print(index)
        except:
            print("Saltó",index)
            faltantes.append(index)
            lista_respuestasOllama.append("NA")
    with open("resultados/rit_base_MultiNLI_DEVM_600_"+str(i+1)+".pickle", "wb") as f:
        pickle.dump(lista_respuestasOllama, f)
    print("Primer rango: ",(i)*600,(i+1)*600)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [19]:
print(len(lista_respuestasOllama))

600
